https://github.com/songys/Chatbot_data

In [66]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools
import math
from torch.jit import script, trace
from torch import optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [67]:
corpus_name = 'chatData'
corpus = os.path.join(r'C:\Users\admin\jupyter\pytorch\chatbot', corpus_name)

def printLines(file, n=10):
    with open(file, 'r', encoding='utf-8') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)
        
printLines(os.path.join(corpus, 'ChatbotData.csv'))

Q,A,label

12시 땡!,하루가 또 가네요.,0

1지망 학교 떨어졌어,위로해 드립니다.,0

3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0

3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0

PPL 심하네,눈살이 찌푸려지죠.,0

SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0

SD카드 안돼,다시 새로 사는 게 마음 편해요.,0

SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0

SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0



In [101]:
def loadLines(fileName, fields):
    lines = {}
    with open(fileName, 'r', encoding='utf-8') as f:
        next(f)
        for line in f:
            values = line.split(',')
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lineObj['text'] = lineObj['Question'] + '\t' + lineObj['Answer']
            # lines[lineObj['text']] = lineObj 
            print(lineObj)
    return lineObj  
     
lines = {}
LINES_FIELDS = ['Question', 'Answer', 'text']
lines = loadLines(os.path.join(corpus, 'ChatbotData.csv'), LINES_FIELDS)
print(len(lines))

{'Question': '12시 땡!', 'Answer': '하루가 또 가네요.', 'text': '12시 땡!\t하루가 또 가네요.'}
{'Question': '1지망 학교 떨어졌어', 'Answer': '위로해 드립니다.', 'text': '1지망 학교 떨어졌어\t위로해 드립니다.'}
{'Question': '3박4일 놀러가고 싶다', 'Answer': '여행은 언제나 좋죠.', 'text': '3박4일 놀러가고 싶다\t여행은 언제나 좋죠.'}
{'Question': '3박4일 정도 놀러가고 싶다', 'Answer': '여행은 언제나 좋죠.', 'text': '3박4일 정도 놀러가고 싶다\t여행은 언제나 좋죠.'}
{'Question': 'PPL 심하네', 'Answer': '눈살이 찌푸려지죠.', 'text': 'PPL 심하네\t눈살이 찌푸려지죠.'}
{'Question': 'SD카드 망가졌어', 'Answer': '다시 새로 사는 게 마음 편해요.', 'text': 'SD카드 망가졌어\t다시 새로 사는 게 마음 편해요.'}
{'Question': 'SD카드 안돼', 'Answer': '다시 새로 사는 게 마음 편해요.', 'text': 'SD카드 안돼\t다시 새로 사는 게 마음 편해요.'}
{'Question': 'SNS 맞팔 왜 안하지ㅠㅠ', 'Answer': '잘 모르고 있을 수도 있어요.', 'text': 'SNS 맞팔 왜 안하지ㅠㅠ\t잘 모르고 있을 수도 있어요.'}
{'Question': 'SNS 시간낭비인 거 아는데 매일 하는 중', 'Answer': '시간을 정하고 해보세요.', 'text': 'SNS 시간낭비인 거 아는데 매일 하는 중\t시간을 정하고 해보세요.'}
{'Question': 'SNS 시간낭비인데 자꾸 보게됨', 'Answer': '시간을 정하고 해보세요.', 'text': 'SNS 시간낭비인데 자꾸 보게됨\t시간을 정하고 해보세요.'}
{'Question': 'SNS보면 나만 빼고 다 행복해보여', 'Answer': '자랑하

{'Question': '주식 망했어', 'Answer': '주식으로 돈버는사람은 몇명 없대요.', 'text': '주식 망했어\t주식으로 돈버는사람은 몇명 없대요.'}
{'Question': '주식으로 돈 많이 벌었어', 'Answer': '주식으로 망하는건 한순간이예요.', 'text': '주식으로 돈 많이 벌었어\t주식으로 망하는건 한순간이예요.'}
{'Question': '주식이 떨어졌어', 'Answer': '주식으로 돈버는사람은 몇명 없대요.', 'text': '주식이 떨어졌어\t주식으로 돈버는사람은 몇명 없대요.'}
{'Question': '주워담지 못할 말을 했어', 'Answer': '두고두고 후회하겠네요.', 'text': '주워담지 못할 말을 했어\t두고두고 후회하겠네요.'}
{'Question': '주제 넘은 짓 한거 같아', 'Answer': '주제를 모를 때가 행복할 때예요.', 'text': '주제 넘은 짓 한거 같아\t주제를 모를 때가 행복할 때예요.'}
{'Question': '주차장에 자리가 없어', 'Answer': '한바퀴만 더 돌면 짠하고 나타날 거예요', 'text': '주차장에 자리가 없어\t한바퀴만 더 돌면 짠하고 나타날 거예요'}
{'Question': '주차장이 꽉 찼어', 'Answer': '한바퀴만 더 돌면 짠하고 나타날 거예요', 'text': '주차장이 꽉 찼어\t한바퀴만 더 돌면 짠하고 나타날 거예요'}
{'Question': '주차하다 긁었어', 'Answer': '속 쓰리겠어요.', 'text': '주차하다 긁었어\t속 쓰리겠어요.'}
{'Question': '주차할데 없어서 짜증나', 'Answer': '한바퀴만 더 돌면 짠하고 나타날 거예요', 'text': '주차할데 없어서 짜증나\t한바퀴만 더 돌면 짠하고 나타날 거예요'}
{'Question': '주택 청약 가입', 'Answer': '내 집 마련의 한 걸음을 내딛으셨네요.', 'text': '주택 청약 가입\t내 집 마련의 한 걸음을 내딛으셨

{'Question': '이젠 놓아줘야 할 때인가 보다', 'Answer': '이별의 끝을 인정하는 것도 용기입니다.', 'text': '이젠 놓아줘야 할 때인가 보다\t이별의 끝을 인정하는 것도 용기입니다.'}
{'Question': '이젠 눈물이 말라버렸나 봅니다.', 'Answer': '많이 지쳤나봐요. 곧 기운낼 수 있길 바랄게요.', 'text': '이젠 눈물이 말라버렸나 봅니다.\t많이 지쳤나봐요. 곧 기운낼 수 있길 바랄게요.'}
{'Question': '이젠 다시 만날 수도 없는 상황', 'Answer': '달라지는게 없다면 이런 상황이 더 나을지도 몰라요.', 'text': '이젠 다시 만날 수도 없는 상황\t달라지는게 없다면 이런 상황이 더 나을지도 몰라요.'}
{'Question': '이젠 더이상 정 주기 무섭습니다', 'Answer': '상처가 많이 크셨나요.', 'text': '이젠 더이상 정 주기 무섭습니다\t상처가 많이 크셨나요.'}
{'Question': '이젠 보내줄게.', 'Answer': '잘 지낼거예요.', 'text': '이젠 보내줄게.\t잘 지낼거예요.'}
{'Question': '이젠 붙잡을 수 도 없는 사람.', 'Answer': '달라지는게 없다면 이런 상황이 더 나을지도 몰라요.', 'text': '이젠 붙잡을 수 도 없는 사람.\t달라지는게 없다면 이런 상황이 더 나을지도 몰라요.'}
{'Question': '이젠 슬픈 사랑', 'Answer': '슬픈 이별이기도 하네요.', 'text': '이젠 슬픈 사랑\t슬픈 이별이기도 하네요.'}
{'Question': '이젠 어떡해얄지 모르겠어', 'Answer': '일기를 써보세요.', 'text': '이젠 어떡해얄지 모르겠어\t일기를 써보세요.'}
{'Question': '이젠 어쩌죠.', 'Answer': '생각을 정리해 보세요.', 'text': '이젠 어쩌죠.\t생각을 정리해 보세요.'}
{'Question': '이젠 유부남이 된 전남친', 'Answer': '

In [72]:
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        for i in range(2):
            inputLine = conversation['Question'][i].strip()
            targetLine = conversation['Answer'][i+1].strip()
            
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    print(qa_pairs)
    return qa_pairs

datafile = os.path.join(corpus, "formatted_lines.txt")

delimiter = '\t'
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(lines):
        writer.writerow(pair)

TypeError: string indices must be integers